# Read Data

In [1]:
# libraries
import os
import networkx as nx

In [ ]:

def plot_graph_with_communities(graph, partition):
    pos = nx.spring_layout(graph)  # Positions for all nodes
    communities = {}
    community_index = 0  # Index for community
    for community in partition.values():
        communities[community_index] = community
        community_index += 1

    colors = [plt.cm.tab20(i / len(communities)) for i in range(len(communities))]

    for community, color in zip(communities.values(), colors):
        nx.draw_networkx_nodes(graph, pos, nodelist=community, node_color=[color])
    nx.draw_networkx_edges(graph, pos)
    nx.draw_networkx_labels(graph, pos)
    plt.show()


In [2]:
def my_read_gml(file_path):
    G = nx.Graph()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    is_node = False
    is_edge = False
    nodes = {}
    edges = set()
    node_id = None

    for line in lines:
        if line.strip().startswith("node"):
            is_node = True
            is_edge = False
            node_id = None
        elif line.strip().startswith("edge"):
            is_node = False
            is_edge = True
            source, target = None, None
        elif is_node:
            if line.strip().startswith("id"):
                node_id = int(line.strip().split()[1])
                nodes[node_id] = {}
        elif is_edge:
            if line.strip().startswith("source"):
                source = int(line.strip().split()[1])
            elif line.strip().startswith("target"):
                target = int(line.strip().split()[1])
            elif line.strip() == "]":
                if source is not None and target is not None:
                    edges.add((source, target))
            else:
                continue

    if min(nodes.keys()) == 0:
        # adaug 1 la toate nodurile si muchiile
        nodes = {k + 1: v for k, v in nodes.items()}
        edges = {(k + 1, v + 1) for k, v in edges}

    G.add_nodes_from(nodes.keys())
    G.add_edges_from(edges)

    return G